In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE146553"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE146553"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE146553.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE146553.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE146553.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE146553_family.soft.gz', 'GSE146553_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Ovarian_Cancer/GSE146553/GSE146553_family.soft.gz
Matrix file: ../../input/GEO/Ovarian_Cancer/GSE146553/GSE146553_series_matrix.txt.gz
Background Information:
!Series_title	"Global DNA hypomethylation in ovarian cancer (Affymetrix_expression_data)"
!Series_summary	"Comparison of DNA methylome, mRNA transcriptome, and copy number variation in tumors with global loss of DNA methylation to tumors with normal global methylation."
!Series_overall_design	"Human tissue samples were obtained from patients undergoing surgical resection at Roswell Park Comprehensive Cancer Center, using Institutional Review Board Protocol I-215512.  Normal controls were obtained from patients without malignancy, and tumor samples contained >90% neoplastic cells.  IOSE-121 and OVCAR3 cells at ~50% confluence were treated with decitabine or PBS vehicle control.  Affymetrix HG 1.0ST arrays and Total RNA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information (Affymetrix_expression_data), this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait: Ovarian Cancer can be inferred from "tissue" field
trait_row = 4  # 'tissue' field contains information about cancer vs normal

# Age: Available in "patient age"
age_row = 2  # 'patient age' field

# Gender: Only one value "female" so it's constant (not useful for association studies)
gender_row = None  # Although gender is available in row 5, it's constant (all female)

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert tissue information to binary trait (0 for normal, 1 for cancer)"""
    if pd.isna(value) or value == "NA":
        return None
    
    # Convert to string if not already
    value = str(value) if not isinstance(value, str) else value
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: 0 for normal, 1 for cancer/tumor
    if "normal" in value.lower():
        return 0
    elif any(cancer_term in value.lower() for cancer_term in ["cancer", "tumor", "carcinoma"]):
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to numeric value"""
    if pd.isna(value) or value == "NA":
        return None
    
    # Convert to string if not already
    value = str(value) if not isinstance(value, str) else value
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to convert to numeric
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the data (Assuming clinical_data was already loaded in previous steps)
    clinical_data = pd.read_csv(f"{in_cohort_dir}/GSE146553_series_matrix.txt.gz", compression='gzip', sep='\t', comment='!', skiprows=0)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=None,  # No need to include gender as it's constant
        convert_gender=None  # No need for gender conversion function
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of clinical data:
{'ID_REF': [nan, 7896740.0], 'GSM4392150': [nan, 8.89], 'GSM4392151': [nan, 6.44], 'GSM4392152': [nan, 5.47], 'GSM4392153': [nan, 8.15], 'GSM4392154': [nan, 5.3], 'GSM4392155': [nan, 5.1], 'GSM4392156': [nan, 6.03], 'GSM4392157': [nan, 6.19], 'GSM4392158': [nan, 6.59], 'GSM4392159': [nan, 6.67], 'GSM4392160': [nan, 5.95], 'GSM4392161': [nan, 5.49], 'GSM4392162': [nan, 5.1], 'GSM4392163': [nan, 4.68], 'GSM4392164': [nan, 4.95], 'GSM4392165': [nan, 5.79], 'GSM4392166': [nan, 6.96], 'GSM4392167': [nan, 4.56], 'GSM4392168': [nan, 5.64], 'GSM4392169': [nan, 5.01], 'GSM4392170': [nan, 6.59], 'GSM4392171': [nan, 4.81], 'GSM4392172': [nan, 5.25], 'GSM4392173': [nan, 5.34], 'GSM4392174': [nan, 16.58], 'GSM4392175': [nan, 6.49], 'GSM4392176': [nan, 6.26], 'GSM4392177': [nan, 4.64], 'GSM4392178': [nan, 6.2], 'GSM4392179': [nan, 4.04], 'GSM4392180': [nan, 6.27], 'GSM4392181': [nan, 5.61], 'GSM4392182': [nan, 5.52], 'GSM4392183': [nan, 6.69], 'GSM4392184': [nan, 5.71], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 29088
First 20 gene/probe identifiers:
Index(['7896736', '7896738', '7896740', '7896742', '7896744', '7896746',
       '7896748', '7896750', '7896752', '7896754', '7896756', '7896759',
       '7896761', '7896779', '7896798', '7896817', '7896822', '7896859',
       '7896861', '7896863'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# The identifiers (e.g., '7896736', '7896738') appear to be numeric IDs, likely probe IDs
# from a microarray platform rather than standard human gene symbols.
# Human gene symbols typically follow patterns like BRCA1, TP53, etc.
# These numeric IDs will need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# From the annotation preview, 'ID' column contains numeric identifiers which match the gene expression data indices
# The 'gene_assignment' column contains gene symbols embedded within longer strings

# 2. Create gene mapping dataframe
# Use get_gene_mapping function with appropriate column names
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("Gene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
print(f"Number of samples: {gene_data.shape[1]}")

# Normalize gene symbols to handle gene synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nAfter gene symbol normalization:")
print(f"Number of genes: {len(gene_data)}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")


Gene mapping preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'Gene': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subfamily G, member 1 pseudogene // --- // ---', 'NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001005484 // OR4F5 // olfactory receptor, family 4, subfamily F, member 5 // 1p36.33 // 79501 /// ENST00000318050 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// ENST00000326183 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// ENST00000335137 // OR4F5 // olfactory receptor, fam

Gene expression data after mapping:
Number of genes: 117461
First 5 gene symbols: ['A-', 'A-3-', 'A-52', 'A-E', 'A-I']
Number of samples: 55

After gene symbol normalization:
Number of genes: 24225
First 5 gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Since we already normalized gene symbols in the previous step (6),
# we can use the gene_data that was already processed

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Since we determined in step 2 that there's no usable trait variation 
# (all samples are cancer cases with no controls), we'll create a clinical dataframe
# but note this limitation

# Create a clinical dataframe with the trait (Ovarian_Cancer)
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create clinical dataframe, but note that all samples have the same trait value
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = 1  # All samples are ovarian cancer tumors

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for ovarian cancer tumors. All samples are tumor samples with no controls, so trait bias is expected and the dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True, 
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable (though we expect it won't be due to trait bias)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to trait bias. Final linked data not saved.")

Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE146553.csv
Sample IDs from gene data: 55 samples
Clinical data shape: (1, 55)
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE146553.csv
Shape of linked data: (55, 24226)
Handling missing values...


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Shape of linked data after handling missing values: (55, 24226)
Checking for bias in features...
Quartiles for 'Ovarian_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Ovarian_Cancer' in this dataset is severely biased.

Dataset validation failed due to trait bias. Final linked data not saved.
